In [44]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

def modify_df(loansData):
    clean_interest_rate = loansData['Interest.Rate'].map(lambda x: round(float(x.rstrip('%')) / 100, 4))
    loansData['Interest.Rate'] = clean_interest_rate
    cleanLoanLength = loansData['Loan.Length'].map(lambda x: int(x.rstrip(' months')))
    loansData['Loan.Length'] = cleanLoanLength
    cleanFICORange = loansData['FICO.Range'].map(lambda x: x.split('-'))
    cleanFICORange = cleanFICORange.map(lambda x: int(x[0]))
    loansData['FICO.Score'] = cleanFICORange
    irtf = loansData['Interest.Rate'] >= .12
    irtf = irtf.astype(int)
    loansData['IR_TF'] = irtf
    loansData = sm.add_constant(loansData)
    loansData2 = loansData
    return loansData2

def define_logistic_model():
    ind_vars = ['const', 'FICO.Score','Amount.Requested']
    logit = sm.Logit(loansData2['IR_TF'], loansData2[ind_vars])
    result = logit.fit()
    coeff = result.params
    return coeff
    
def logistic_function(coeff, score, amount):
    getLoan = 1/ (1 + np.exp(-(coeff[0] + coeff[1] * score + coeff[2] * amount)))
    return getLoan

loansData = pd.read_csv('https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv')
loansData2 = modify_df(loansData)
loansData2.to_csv('loansData_clean.csv', header=True, index=False)

coeff = define_logistic_model()

x = np.linspace(min(loansData2['FICO.Score']), min(loansData2['FICO.Score']), 500)
y = np.linspace(min(loansData2['Amount.Requested']), min(loansData2['Amount.Requested']), 500)

'''fico_IRTF0 = loansData2.loc[loansData2['IR_TF']==0, 'FICO.Score']
fico_IRTF1 = loansData2.loc[loansData2['IR_TF']==1, 'FICO.Score']

loanamt_IRTF0 = loansData2.loc[loansData2['IR_TF']==0, 'Amount.Requested']
loanamt_IRTF1 = loansData2.loc[loansData2['IR_TF']==1, 'Amount.Requested']'''

plt.plot(x,logistic_function(coeff,x,y))

plt.show()

Optimization terminated successfully.
         Current function value: 0.319503
         Iterations 8
